In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import torch
import torchvision
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [6]:
path = "/kaggle/input/pizza-not-pizza/pizza_not_pizza/"

In [15]:
a = cv2.imread(path + "not_pizza/503742.jpg")
a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
Image.fromarray(a)

In [13]:
a[:,:,2]

In [12]:
a.shape

In [13]:
import os

NP = os.listdir(path + "not_pizza")
TP = os.listdir(path + "pizza")

print("Not Pizza 데이터 개수 : ", len(NP))
print("Pizza 데이터 개수 : ", len(TP))

In [14]:
import random

b = cv2.imread(path + "pizza/" + random.sample(TP,1)[0])
b = cv2.cvtColor(b, cv2.COLOR_BGR2RGB)
Image.fromarray(b)

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# Make Dataset

In [16]:
from torch.utils.data import TensorDataset, Dataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from torchvision import datasets, transforms

In [17]:
cv2.imread(path + "pizza/" + random.sample(TP,1)[0])

In [18]:
data_transform = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.486, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.486, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [19]:
data_transform["train"]

In [20]:
images_dataset = {x : datasets.ImageFolder(root = path, transform = data_transform[x]) for x in ['train', 'val']}
images_dataset

In [21]:
dataset_size = {x : len(images_dataset[x]) for x in ["train", "val"]}
dataset_size

In [22]:
images_dataset["train"].classes

In [23]:
train_dataloader = DataLoader(images_dataset['train'], batch_size= 4, shuffle=True, num_workers=4)
val_dataloader = DataLoader(images_dataset['val'], batch_size= 4, shuffle=True, num_workers=4)

In [24]:
def imageloader(name, device, training = False):
    image = Image.open(name).convert("RGB")
    image = transforms.Resize((256,256))(image)
    
    if training :
        image = data_transform["train"](image)
    else :
        imgae = data_transform["val"](image)
        
    return image.unsqueeze(0).to(device, torch.float)

In [25]:
class LoadDataset(Dataset):
    def __init__(self, image2label, device, training = False):
        self.images = list(image2label.key())
        self.labels = list(image2label.value())
        self.training = training
    
    def __getitem__(self, idx):
        try:
            image_name = self.images[idx]
            img = imageloader(image_name, self.device, training = self.training)
            
            label = self.labels[idx]
            label = torch.from_numpy(np.array(label, dtype = np.float32)).to(self.device)
        except exception as ex:
            return None
        
    def __len__(self):
        return len(self.images)
                

# Modeling

In [26]:
b.shape

In [52]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding =1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc = torch.nn.Linear(56 * 56 * 64, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [53]:
model = CNN().to(device)

In [29]:
batch_size = 12
lr = 0.002
epochs = 100

In [30]:
loss_fun = nn.CrossEntropyLoss()

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [ ]:
loss_arr = []
for i in range(epochs):
    for j, [image,label] in enumerate(train_dataloader):
        x = image.to(device)
        y = label.to(device)
        
        optimizer.zero_grad()
        
        output = model.forward(x)
        
        loss = loss_fun(output, y)
        loss.backward()
        
        optimizer.step()
        
        if j % 10 == 0 :
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())        